![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 8. Generic Classifier v3.0

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark


sparknlp_jsl.version : 3.0.1
Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.164.233.65:7077 
 AppName 
 Databricks Shell

## Load dataset

In [0]:
 !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv
 
 dbutils.fs.cp("file:/databricks/driver/petfinder-mini.csv", "dbfs:/") 

Out[3]: True

In [0]:
dataframe = pd.read_csv('petfinder-mini.csv')

In [0]:
# In the original dataset "4" indicates the pet was not adopted.

dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [0]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [0]:
dataframe.head()

Out[7]:

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of ca...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to p...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in S...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for m...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our pupp...,3,1


In [0]:
dataframe.columns

Out[8]: Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
 'PhotoAmt', 'target'],
 dtype='object')

In [0]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Type 11537 non-null object
 1 Age 11537 non-null int64 
 2 Breed1 11537 non-null object
 3 Gender 11537 non-null object
 4 Color1 11537 non-null object
 5 Color2 11537 non-null object
 6 MaturitySize 11537 non-null object
 7 FurLength 11537 non-null object
 8 Vaccinated 11537 non-null object
 9 Sterilized 11537 non-null object
 10 Health 11537 non-null object
 11 Fee 11537 non-null int64 
 12 Description 11528 non-null object
 13 PhotoAmt 11537 non-null int64 
 14 target 11537 non-null int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB

In [0]:
dataframe.target.value_counts()

Out[10]: 1 8457
0 3080
Name: target, dtype: int64

In [0]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(handle_unknown='ignore', categories='auto'), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [0]:
y.nunique()


Out[13]: 2

In [0]:
X.shape

Out[14]: (11537, 302)

In [0]:
input_dim = X.shape[1]

In [0]:
input_dim

Out[16]: 302

In [0]:
import scipy.sparse

df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df = df.fillna(0)

df['target']= y

df.head()

Out[17]:

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_253,col_254,col_255,col_256,col_257,col_258,col_259,col_260,col_261,col_262,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.171825,0.0,0.137765,0.0,0.000000,0.283609,0.0,0.220992,0.151594,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.144907,0.000000,0.0,0.158869,0.0,0.0,0.000000,0.170202,0.000000,0.192465,0.190783,0.000000,0.132427,0.176639,0.151246,0.000000,0.0,0.000000,0.000000,0.257843,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.228484,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.256152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250843,0.000000,0.0,0.000000,0.000000,0.213817,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.141649,0.000000,0.0,0.000000,0.0,0.172449,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.307225,0.0,0.178193,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.210999,0.209155,0.209619,0.000000,0.000000,0.414527,0.179294,0.0,0.000000,0.000000,0.141336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.257886,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.198538,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.126763,0.113263,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.081638,0.000000,0.105362,0.128186,0.0,0.000000,0.0,0.000000,0.107062,0.086649,0.0,0.000000,0.0,0.0,0.000000,0.125751,0.218471,0.142200,0.000000,0.000000,0.195683,0.000000,0.111746,0.120832,0.0,0.120993,0.000000,0.000000,0.223069,0.117846,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [0]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Sparse pandas data (column col_0) not supported.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
col_293|col_294|col_295|col_296|col_297|col_298| col_299| col_300| col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|-0.4524794726808656| 0.9502875792756131|-0.8297616989552165| 1|
 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844| 1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows

In [0]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9190
Test Dataset Count: 2347

## Create a custom DL architecture with TF

In [0]:
%sh cd /databricks/python/lib/python3.7/site-packages/sparknlp_jsl/_tf_graph_builders/ && ls -la

total 40
drwxr-xr-x 7 root root 4096 May 1 11:36 .
drwxr-xr-x 4 root root 4096 May 1 11:36 ..
-rw-r--r-- 1 root root 0 May 1 11:36 __init__.py
drwxr-xr-x 2 root root 4096 May 1 11:36 __pycache__
drwxr-xr-x 3 root root 4096 May 1 11:36 assertion_dl
drwxr-xr-x 3 root root 4096 May 1 11:36 generic_classifier
-rw-r--r-- 1 root root 9452 May 1 11:36 graph_builders.py
drwxr-xr-x 3 root root 4096 May 1 11:36 ner_dl
drwxr-xr-x 3 root root 4096 May 1 11:36 text2sql

In [0]:
import tensorflow as tf

tf.__version__

Out[21]: '1.15.0'

In [0]:
from sparknlp_jsl.training import tf_graph

#!mkdir gc_graph

tf_graph.print_model_params("generic_classifier")


generic_classifier parameters.
Parameter Required Default value Description
hidden_layers no [200] List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim yes - Input dimension.
output_dim yes - Output dimension.
hidden_act no relu Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2 no 0 L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2 no 0 L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm no 0 Batch normalization. Boolean (0 or 1).

In [0]:
DL_params = {"input_dim": input_dim, 
             "output_dim": y.nunique(), 
             "hidden_layers": [300, 200, 100], 
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


tf_graph.build("generic_classifier",build_params=DL_params, model_location="file:/databricks/driver/gc_graph", model_filename="auto")

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/losses/losses_impl.py:121: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
generic_classifier graph exported to file:/databricks/driver/gc_graph/gcl.302.2.pb

In [0]:
%sh pwd && ls -la

/databricks/driver
total 9336
drwxr-xr-x 1 root root 4096 May 1 12:00 .
drwxr-xr-x 1 root root 4096 May 1 11:35 ..
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
drwxr-xr-x 3 root root 4096 May 1 11:35 eventlogs
drwxr-xr-x 3 root root 4096 May 1 11:47 file:
drwxr-xr-x 2 root root 4096 May 1 12:00 ganglia
drwxr-xr-x 2 root root 4096 May 1 12:01 logs
-rw-r--r-- 1 root root 4763125 May 1 11:44 petfinder-mini.csv
-rw-r--r-- 1 root root 4763125 May 1 11:55 petfinder-mini.csv.1

In [0]:
#or just use the one we already have in the repo
#! mkdir /databricks/driver/gc_graph

#!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /databricks/driver/gc_graph

In [0]:
%sh cd file:/databricks/driver/gc_graph && ls -la

total 144
drwxr-xr-x 2 root root 4096 May 1 12:03 .
drwxr-xr-x 3 root root 4096 May 1 11:47 ..
-rw-r--r-- 1 root root 135651 May 1 12:03 gcl.302.2.pb

In [0]:
from sparknlp_jsl.base import *

#!mkdir logs

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('file:/databricks/driver/gc_graph/gcl.302.2.pb')\
    .setEpochsNumber(100)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])



In [0]:
# train 100 epochs (takes around 37 seconds)

clf_model = clf_Pipeline.fit(training_data)


In [0]:
%sh cd logs && ls -lt

total 1016
-rw-r--r-- 1 root root 55754 May 1 12:06 active.log
-rw-r--r-- 1 root root 48676 May 1 12:06 log4j-active.log
-rw-r--r-- 1 root root 777436 May 1 12:06 usage.json
-rw-r--r-- 1 root root 7766 May 1 12:06 GenericClassifierApproach_76182a838ccc.log
-rw-r--r-- 1 root root 10596 May 1 12:06 stdout
-rw-r--r-- 1 root root 2815 May 1 12:05 stderr
-rw-r--r-- 1 root root 126620 May 1 12:05 metrics.json

In [0]:
%sh

cat logs/GenericClassifierApproach_76182a838ccc.log

Training 100 epochs
Epoch 1/100	0.61s	Loss: 18.505451	ACC: 0.6382848	Validation ACC: 0.63656145
Epoch 2/100	0.25s	Loss: 16.11135	ACC: 0.714522	Validation ACC: 0.6648531
Epoch 3/100	0.26s	Loss: 14.842516	ACC: 0.74587315	Validation ACC: 0.6702938
Epoch 4/100	0.26s	Loss: 13.1608715	ACC: 0.7763724	Validation ACC: 0.6844396
Epoch 5/100	0.26s	Loss: 11.499329	ACC: 0.81780654	Validation ACC: 0.6985854
Epoch 6/100	0.25s	Loss: 9.851777	ACC: 0.8558212	Validation ACC: 0.6980413
Epoch 7/100	0.24s	Loss: 8.017818	ACC: 0.88914746	Validation ACC: 0.70293796
Epoch 8/100	0.23s	Loss: 6.2894115	ACC: 0.9198023	Validation ACC: 0.71653974
Epoch 9/100	1.37s	Loss: 4.677906	ACC: 0.9450415	Validation ACC: 0.711099
Epoch 10/100	0.24s	Loss: 3.6929731	ACC: 0.9573285	Validation ACC: 0.7176279
Epoch 11/100	0.23s	Loss: 2.8234046	ACC: 0.97148645	Validation ACC: 0.7040261
Epoch 12/100	0.24s	Loss: 2.2184603	ACC: 0.9783995	Validation ACC: 0.71653974
Epoch 13/100	0.24s	Loss: 1.500175	ACC: 0.9879732	Validation ACC: 0.72687703
Epoch 14/100	0.23s	Loss: 1.2979617	ACC: 0.9890647	Validation ACC: 0.7279652
Epoch 15/100	0.31s	Loss: 1.1846932	ACC: 0.99	Validation ACC: 0.7328618
Epoch 16/100	0.29s	Loss: 0.8029058	ACC: 0.99297315	Validation ACC: 0.72687703
Epoch 17/100	0.28s	Loss: 0.8454745	ACC: 0.9933785	Validation ACC: 0.7236126
Epoch 18/100	0.24s	Loss: 0.6468263	ACC: 0.99418926	Validation ACC: 0.7236126
Epoch 19/100	0.24s	Loss: 0.68013835	ACC: 0.9939294	Validation ACC: 0.72252446
Epoch 20/100	0.25s	Loss: 0.70406216	ACC: 0.99324334	Validation ACC: 0.72850925
Epoch 21/100	0.23s	Loss: 1.0516644	ACC: 0.9878902	Validation ACC: 0.7257889
Epoch 22/100	0.24s	Loss: 1.4800633	ACC: 0.98272365	Validation ACC: 0.7198041
Epoch 23/100	0.24s	Loss: 0.974042	ACC: 0.99001044	Validation ACC: 0.71653974
Epoch 24/100	0.23s	Loss: 0.68933797	ACC: 0.9924429	Validation ACC: 0.7149075
Epoch 25/100	0.24s	Loss: 0.46592456	ACC: 0.9959564	Validation ACC: 0.72306854
Epoch 26/100	0.23s	Loss: 0.34179324	ACC: 0.99662185	Validation ACC: 0.73612624
Epoch 27/100	0.24s	Loss: 0.32605368	ACC: 0.9967672	Validation ACC: 0.7334059
Epoch 28/100	0.23s	Loss: 0.3866503	ACC: 0.99554074	Validation ACC: 0.72850925
Epoch 29/100	0.24s	Loss: 0.32163975	ACC: 0.99621624	Validation ACC: 0.72633296
Epoch 30/100	0.23s	Loss: 0.2719338	ACC: 0.99729735	Validation ACC: 0.73231775
Epoch 31/100	0.23s	Loss: 0.21317704	ACC: 0.99770296	Validation ACC: 0.73014146
Epoch 32/100	0.24s	Loss: 0.18228042	ACC: 0.99797314	Validation ACC: 0.7257889
Epoch 33/100	0.23s	Loss: 0.2902529	ACC: 0.99716216	Validation ACC: 0.72687703
Epoch 34/100	0.24s	Loss: 0.2036954	ACC: 0.99756765	Validation ACC: 0.7350381
Epoch 35/100	0.24s	Loss: 0.24387403	ACC: 0.996892	Validation ACC: 0.7295974
Epoch 36/100	0.24s	Loss: 0.2681201	ACC: 0.99756765	Validation ACC: 0.73068553
Epoch 37/100	0.24s	Loss: 0.18177333	ACC: 0.997838	Validation ACC: 0.7257889
Epoch 38/100	0.23s	Loss: 0.17030343	ACC: 0.9981082	Validation ACC: 0.7350381
Epoch 39/100	0.23s	Loss: 0.3672492	ACC: 0.99622667	Validation ACC: 0.7252448
Epoch 40/100	0.23s	Loss: 1.2548143	ACC: 0.98745364	Validation ACC: 0.71273124
Epoch 41/100	0.23s	Loss: 4.0241666	ACC: 0.9513718	Validation ACC: 0.70674646
Epoch 42/100	0.25s	Loss: 2.1984212	ACC: 0.9720582	Validation ACC: 0.7149075
Epoch 43/100	0.24s	Loss: 0.7176924	ACC: 0.9925677	Validation ACC: 0.7132753
Epoch 44/100	0.24s	Loss: 0.3439331	ACC: 0.9970271	Validation ACC: 0.72306854
Epoch 45/100	0.23s	Loss: 0.30007225	ACC: 0.9977027	Validation ACC: 0.7214363
Epoch 46/100	0.24s	Loss: 0.1812745	ACC: 0.997838	Validation ACC: 0.7274211
Epoch 47/100	0.23s	Loss: 0.15850109	ACC: 0.9981082	Validation ACC: 0.72470075
Epoch 48/100	0.23s	Loss: 0.1589234	ACC: 0.99852383	Validation ACC: 0.72633296
Epoch 49/100	0.23s	Loss: 0.21003477	ACC: 0.9979731	Validation ACC: 0.72850925
Epoch 50/100	0.23s	Loss: 0.16109736	ACC: 0.99837834	Validation ACC: 0.7241567
Epoch 51/100	0.23s	Loss: 0.110399716	ACC: 0.99837846	Validation ACC: 0.72470075
Epoch 52/100	0.23s	Loss: 0.17128992	ACC: 0.9978482	Validation A

In [0]:
pred_df = clf_model.transform(test_data)

In [0]:
pred_df.select('target','prediction.result').show()

+------+------+
target|result|
+------+------+
 1| [1]|
 1| [1]|
 1| [1]|
 0| [1]|
 0| [0]|
 1| [1]|
 0| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [0]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 0| [0]|
+------+------+
only showing top 20 rows

In [0]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [0]:
preds_df

Out[61]:

,target,result
0,1,1
1,1,1
2,1,1
3,0,1
4,0,0
...,...,...
2342,1,1
2343,0,1
2344,1,0
2345,1,1


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['target'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['target']))


precision recall f1-score support

 0 0.4418 0.5073 0.4723 546
 1 0.8436 0.8057 0.8242 1801

 accuracy 0.7363 2347
 macro avg 0.6427 0.6565 0.6482 2347
weighted avg 0.7501 0.7363 0.7423 2347

0.7362590541116318

# Case Study: Alexa Review Classification

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv

In [0]:
import pandas as pd
df = pd.read_csv('amazon_alexa.tsv', sep='\t')
df

Out[38]:

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wr...",1
3,5,31-Jul-18,Charcoal Fabric,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the light...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in between!!",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, checking time, looking up weather. There are many more ...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my entire home, TV, all my lights, my thermostat, my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound quality isn't great. I mostly use it for commands though...,1


In [0]:
df.verified_reviews = df.verified_reviews.str.lower()

In [0]:
df.feedback.value_counts()

Out[41]: 1 2893
0 257
Name: feedback, dtype: int64

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [0]:
import scipy.sparse

sdf = pd.DataFrame.sparse.from_spmatrix(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf = sdf.fillna(0)

sdf['feedback']= y

sdf.head()

Out[44]:

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_951,col_952,col_953,col_954,col_955,col_956,col_957,col_958,col_959,col_960,col_961,col_962,col_963,col_964,col_965,col_966,col_967,col_968,col_969,col_970,col_971,col_972,col_973,col_974,col_975,col_976,col_977,col_978,col_979,col_980,col_981,col_982,col_983,col_984,col_985,col_986,col_987,col_988,col_989,col_990,col_991,col_992,col_993,col_994,col_995,col_996,col_997,col_998,col_999,feedback
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145963,0.146283,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.327234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324472,0.0,0.149662,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.16371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104466,0.0,0.0,0.0,0.0,0.0,0.210616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [0]:
input_spark_df = spark.createDataFrame(sdf)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Sparse pandas data (column col_0) not supported.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2482
Test Dataset Count: 668

In [0]:
%sh cd file:/databricks/driver/gc_graph && ls -la

total 144
drwxr-xr-x 2 root root 4096 May 1 12:03 .
drwxr-xr-x 3 root root 4096 May 1 11:47 ..
-rw-r--r-- 1 root root 135651 May 1 12:03 gcl.302.2.pb

In [0]:
from sparknlp_jsl.base import *

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\
   #.setModelFile("/databricks/driver/gc_graph/pet_in1202D_out2.pb")
    

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])

clf_model = clf_Pipeline.fit(training_data)


In [0]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['feedback'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['feedback']))


precision recall f1-score support

 0 0.6364 0.4444 0.5234 63
 1 0.9439 0.9736 0.9585 605

 accuracy 0.9237 668
 macro avg 0.7901 0.7090 0.7409 668
weighted avg 0.9149 0.9237 0.9175 668

0.9236526946107785